<a href="https://colab.research.google.com/github/thq4n/Learn-Colab/blob/main/DLGB_Building_a_custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://sourceforge.net/projects/fotohound/files/sample-pictures/Sample/Sample-Pictures.zip
!unzip Sample-Pictures.zip
!mv "Sample Pictures" train

--2021-05-02 08:14:08--  https://sourceforge.net/projects/fotohound/files/sample-pictures/Sample/Sample-Pictures.zip
Resolving sourceforge.net (sourceforge.net)... 216.105.38.13
Connecting to sourceforge.net (sourceforge.net)|216.105.38.13|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://sourceforge.net/projects/fotohound/files/sample-pictures/Sample/Sample-Pictures.zip/ [following]
--2021-05-02 08:14:08--  https://sourceforge.net/projects/fotohound/files/sample-pictures/Sample/Sample-Pictures.zip/
Reusing existing connection to sourceforge.net:443.
HTTP request sent, awaiting response... 302 Found
Location: https://sourceforge.net/projects/fotohound/files/sample-pictures/Sample/Sample-Pictures.zip/download [following]
--2021-05-02 08:14:08--  https://sourceforge.net/projects/fotohound/files/sample-pictures/Sample/Sample-Pictures.zip/download
Reusing existing connection to sourceforge.net:443.
HTTP request sent, awaiting response... 302

In [2]:
import torch
import torchvision
import cv2
from PIL import Image
import matplotlib.pyplot as plt

import os
import shutil
import random; random.seed(1)

In [ ]:
labels = ['Flowers', 'Scenery', 'Zoo']

train_directory = './train'
val_directory = './val'

try:
    os.mkdir(val_directory)
except FileExistsError:
    pass

for label in labels:
    os.mkdir(os.path.join(val_directory, label))
    
    images = os.listdir(os.path.join(train_directory, label))
    val_images = random.sample(images, len(images) // 5)
    
    for image in val_images:
        shutil.move(
            os.path.join(train_directory, label, image),
            os.path.join(val_directory, label, image)
        )


In [ ]:
sample_train_img = cv2.imread('train/Flowers/DSC_0020_th.jpg')
plt.imshow(sample_train_img)
plt.axis('off')
plt.show()

In [ ]:
sample_val_img = cv2.imread('val/Zoo/DSC_4351.jpg')
plt.imshow(sample_val_img)
plt.axis('off')
plt.show()

In [ ]:
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, directory, transform=None):
        self.directory = directory
        self.dataset = torchvision.datasets.ImageFolder(
            self.directory,
            transform=transform,
            loader=self.__load_image
        )
        self.transform = transform
    
    def __len__(self):
        return len(self.dataset)
    
    def __load_image(self, path):
        img = cv2.imread(path, 0)
        img = cv2.resize(img, (500, 500))
        img = Image.fromarray(img, mode='L')
        return img
    
    def __getitem__(self, idx):
        return self.dataset[idx]

In [ ]:
train_dataset = ImageDataset(train_directory)
val_dataset = ImageDataset(val_directory)

In [ ]:
n_img = 8

for i in range(n_img // 2):
    plt.subplot(2, 4, i + 1)
    img, label = train_dataset[random.randint(0, len(train_dataset) - 1)]
    plt.imshow(img, cmap='gray')
    plt.title(train_dataset.dataset.classes[label])
    plt.axis('off')

for i in range(n_img // 2):
    plt.subplot(2, 4, n_img // 2 + i + 1)
    img, label = val_dataset[random.randint(0, len(val_dataset) - 1)]
    plt.imshow(img, cmap='gray')
    plt.title(val_dataset.dataset.classes[label])
    plt.axis('off')

plt.show()